# Build Chains and Results 

In [1]:
import subprocess
data_file_name = 'EMS_DCMA_DD_23.08.graphml'
experiment = None
tasks_types = 'tdas' # 'milestones' for milestones only, not yet available
results = 'prt'# 'chains' for a list of chains only, not yet available
query = 'code' # 'web' if the user intends to query the results via the web shell as well
if not experiment: experiment = data_file_name.replace('.graphml', '').replace('.', '_')
from calculate import *
run_calculation_process(data_file_name, experiment, tasks_types, results, query)

Starting EC2 instance: i-0586e11281d4b02a2
EC2 instance "i-0586e11281d4b02a2" has been started
Data file uploaded to S3
Run attempt encountered error:
 []
Calculation finished
Compute instance stopped
Preparing results
The results have been downloaded to EMS_DCMA_DD_23_08


# Query Results  

In [2]:
import duckdb
con = duckdb.connect()
results_files = "'{e}/*.parquet'".format(e=experiment)
query = """SELECT * FROM {rf} """.format(rf=results_files)
query_df = con.execute(query).df()
#query_df.to_excel('query_df.xlsx')
display(query_df.head(20))

,ID,ChainID,TaskID,NeighbourID,Dependency,TaskType,Label,PlannedStart,PlannedEnd,ActualStart,ActualEnd,Float1,Status,File,planned_duration,actual_duration
0,MI20-EMS-N2-60000,C1,C1T1,MI20-EMS-N2-93640,FS,TT_FinMile,JS0 - T0 TF - NTP,2021-06-30,2021-06-30,2021-06-30,2021-06-30,0,TK_Complete,EMS_DCMA_DD_23.08.graphml,0,0.0
1,MI20-EMS-N2-93640,C1,C1T2,MI20-EMS-N2-93650,FS,TT_Task,Activits SDR Integration [ex-CDR I],2021-01-07,2021-01-07,2021-01-07,2021-12-15,0,TK_Complete,EMS_DCMA_DD_23.08.graphml,0,167.0
2,MI20-EMS-N2-93650,C1,C1T3,MI20-EMS-N2-93660,FS,TT_FinMile,OIM - SDR Integration [ex-CDR I],2021-12-15,2021-12-15,2021-12-15,2021-12-15,0,TK_Complete,EMS_DCMA_DD_23.08.graphml,0,0.0
3,MI20-EMS-N2-93660,C1,C1T4,MI20-EMS-N2-93670,FS,TT_Task,Activits IDR Integration,2021-12-16,2021-12-16,2021-12-16,nan,0,TK_Active,EMS_DCMA_DD_23.08.graphml,0,nan
4,MI20-EMS-N2-93670,C1,C1T5,MI20-EMS-N2-61080,FS,TT_FinMile,OIM - IDR Integration,2022-01-09,2022-01-09,nan,nan,0,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan
5,MI20-EMS-N2-61080,C1,C1T6,MI20-EMS-N2-61090,FS,TT_FinMile,OIM - IDRs - Interface Gate [TRAIN 1],2023-02-01,2023-02-01,nan,nan,0,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan
6,MI20-EMS-N2-61090,C1,C1T7,MI20-EMS-N2-61110,FS,TT_Mile,IIM - IGR - Interface Gate - OFFICIEL,2022-10-27,2022-10-27,nan,nan,0,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan
7,MI20-EMS-N2-61110,C1,C1T8,MI20-EMS-N2-121078,FS,TT_Mile,IIM - PGR - Preliminary Gate - OFFICIEL,2023-07-04,2023-07-04,nan,nan,0,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan
8,MI20-EMS-N2-121078,C1,C1T9,MI20-EMS-N2-121108,FS,TT_Mile,IIM - PGR - Preliminary Gate [Train Design] - OFFICIEL,2023-07-04,2023-07-04,nan,nan,0,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan
9,MI20-EMS-N2-121108,C1,C1T10,MI20-EMS-N2-121138,FS,TT_Mile,IIM - CGR Full - Full Critical Gate [Train Design] - OFFICIEL,2023-08-08,2023-08-08,nan,nan,0,TK_NotStart,EMS_DCMA_DD_23.08.graphml,0,nan


In [ ]:
len(query_df)